In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from itertools import permutations
import random
from datetime import datetime

print(tf.__version__)

1.13.1


In [2]:
img_width = 224
img_height = 224

In [3]:
tfrecord_train = 'facenet_train.tfrecord'
#tfrecord_test = 'facenet_test.tfrecord'
tfrecord_dir = 'tfrecords'

print(os.getcwd())
if not os.path.exists(tfrecord_dir):
    os.makedirs(tfrecord_dir)

C:\Users\Insup\Desktop\ForJupyter\SimpleFaceNet


In [4]:
# 사람 이름 뽑아내기
path = 'd:\\facenet_data\\newImage'
pic_path = os.path.join(os.getcwd(), 'newImage')
anc_pos_path = os.path.join(os.getcwd(), 'newImage')
test_path = os.path.join(os.getcwd(), 'newImage')
print(os.path.exists(path))
print(os.path.exists(pic_path))
print(os.path.exists(anc_pos_path))
print(os.path.exists(test_path))

True
False
False
False


In [5]:
whole_categories = sorted(os.listdir(path))
anc_pos_categories = sorted(os.listdir(path))
test_categories = sorted(os.listdir(path))
print(anc_pos_categories)

['0000045', '0000099', '0000100', '0000102', '0000103', '0000105', '0000107', '0000108', '0000114', '0000117', '0000119', '0000121', '0000133', '0000137', '0000141', '0000143', '0000144', '0000145', '0000147', '0000156', '0000157', '0000159', '0000166', '0000168', '0000169', '0000170', '0000174', '0000177', '0000183', '0000185', '0000186', '0000188', '0000189', '0000192', '0000195', '0000198', '0000202', '0000204', '0000205', '0000207', '0000208', '0000210', '0000211', '0000212', '0000214', '0000220', '0000225', '0000233', '0000238', '0000240', '0000247', '0000249', '0000254', '0000256', '0000260', '0000262', '0000263', '0000268', '0000270', '0000271', '0000272', '0000275', '0000280', '0000281', '0000282', '0000284', '0000286', '0000287', '0000293', '0000295', '0000296', '0000297', '0000299', '0000301', '0000302', '0000304', '0000307', '0000310', '0000317', '0000318', '0000319', '0000321', '0000324', '0000327', '0000331', '0000332', '0000333', '0000334', '0000335', '0000342', '0000343'

In [6]:
anc_pos_set = []
neg_set = []
test_set = []
label_set = []
s = datetime.now().strftime('%H:%M:%S')
num_count = 0
print(s)
for category in anc_pos_categories:
    if len(os.listdir(os.path.join(path, category)))<2:
        continue;
    for _ in range(8):
        #print(os.listdir(os.path.join(anc_pos_path, category)))
        
        rnd_list1 = random.sample(os.listdir(os.path.join(path, category)), 2)
        rnd_list1[0] = os.path.join(path, category, rnd_list1[0])
        rnd_list1[1] = os.path.join(path, category, rnd_list1[1])
        #print(rnd_list1)
        #temp = random.sample(os.listdir(os.path.join(anc_pos_path, category)), 2)
        temp_list = whole_categories.copy()
        temp_list.remove(category)
        ran_list = random.sample(temp_list, 20)
        container_list = []
        for neg_category in ran_list:
            rnd_list = random.sample(os.listdir(os.path.join(path, neg_category)), 1)
            rnd_list[0] = os.path.join(path, neg_category, rnd_list[0])
            container_list.append(rnd_list[0])

        #flatten 2D -> 1D
        #container_list = sum(container_list, [])
        #print(container_list)

        anc_pos_set.append(rnd_list1)
        neg_set.append(container_list)
        num_count += 1
        if num_count%50==0:
            print(num_count)

'''
for category in test_categories:    
    test_list = os.listdir(os.path.join(test_path, category))
    for file in test_list:
        file_path = os.path.join(pic_path, category, file)
        print(file_path)
        test_set.append(file_path)
        label_set.append(category)
'''
s = datetime.now().strftime('%H:%M:%S')
print(s)
print(num_count)

# print(len(anc_pos_set))
# print(len(neg_set))
# print(anc_pos_set[0])
# print(neg_set[0])    
    

10:04:12
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
10050
1010

70100
70150
70200
70250
70300
70350
70400
70450
70500
70550
70600
70650
70700
70750
70800
70850
70900
70950
71000
71050
71100
71150
71200
71250
71300
71350
71400
71450
71500
71550
71600
71650
71700
71750
71800
71850
71900
71950
72000
72050
72100
72150
72200
72250
72300
72350
72400
72450
72500
72550
72600
72650
72700
72750
72800
72850
72900
72950
73000
73050
73100
73150
73200
73250
73300
73350
73400
73450
73500
73550
73600
73650
73700
73750
73800
73850
73900
73950
74000
74050
74100
74150
74200
74250
74300
74350
74400
74450
74500
74550
74600
74650
74700
74750
74800
74850
74900
74950
75000
75050
75100
75150
75200
75250
75300
75350
75400
75450
75500
75550
75600
75650
75700
75750
75800
75850
75900
75950
76000
76050
76100
76150
76200
76250
76300
76350
76400
76450
76500
76550
76600
76650
76700
76750
76800
76850
76900
76950
77000
77050
77100
77150
77200
77250
77300
77350
77400
77450
77500
77550
77600
77650
77700
77750
77800
77850
77900
77950
78000
78050
78100
78150
78200
78250
78300
78350
7840

In [7]:
tfpath = 'd:\\facenet_data\\facenet_tfrecord'
# tfrecord file 경로 설정
train_tfr_path = os.path.join(tfpath, tfrecord_train)
print(train_tfr_path)

# tfrecord writer
writer_train = tf.python_io.TFRecordWriter(train_tfr_path)
'''
#test file tfrecord 경로 설정
test_tfr_path = os.path.join(os.getcwd(), tfrecord_dir, tfrecord_test)

# tfrecord_test writer
writer_test = tf.python_io.TFRecordWriter(test_tfr_path)
'''

d:\facenet_data\facenet_tfrecord\facenet_train.tfrecord


'\n#test file tfrecord 경로 설정\ntest_tfr_path = os.path.join(os.getcwd(), tfrecord_dir, tfrecord_test)\n\n# tfrecord_test writer\nwriter_test = tf.python_io.TFRecordWriter(test_tfr_path)\n'

In [8]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [9]:
index_list = [i for i in range(len(anc_pos_set))]
index_list = np.random.permutation(index_list)

index_list_for_test = [i for i in range(len(test_set))]
index_list_for_test = np.random.permutation(index_list_for_test)

num_count = 0

# training set을 tfrecord로 저장
for index in index_list:
    
    num_count += 1
    if num_count%100==0:
        print(num_count)
    
    anchor_path = anc_pos_set[index][0]
    #print(anchor_path)
    pos_path = anc_pos_set[index][1]
    neg_path_bundle = neg_set[index]
    
    anc_image = Image.open(anchor_path)
    anc_image = anc_image.resize((img_height, img_width))
    anc_image = np.asarray(anc_image)
    
    pos_image = Image.open(pos_path)
    pos_image = pos_image.resize((img_height, img_width))
    pos_image = np.asarray(pos_image)
    
    neg_list = []
    for neg_path in neg_path_bundle:
        neg_image = Image.open(neg_path)
        neg_image = neg_image.resize((img_height, img_width))
        neg_image = np.asarray(neg_image)
        #neg_image_str = neg_image.tobytes()
        neg_list.append(neg_image)
    neg_list = np.asarray(neg_list)
    #print(np.shape(neg_list))
    anc_image_str = anc_image.tobytes()
    pos_image_str = pos_image.tobytes()
    neg_image_str = neg_list.tobytes()
    
    example = tf.train.Example(features=tf.train.Features(feature={
        'anc_image' : _bytes_feature(anc_image_str),
        'pos_image' : _bytes_feature(pos_image_str),
        'neg_image_bundle' : _bytes_feature(neg_image_str)
    }))
    
    writer_train.write(example.SerializeToString())
'''

for index in index_list_for_test:
    test_path = test_set[index]
    
    test_image = Image.open(test_path)
    test_image = test_image.resize((img_height, img_width))
    test_image = np.asarray(test_image)
    test_image_str = test_image.tobytes()
    
    example = tf.train.Example(features=tf.train.Features(feature={
        'test_image' : _bytes_feature(test_image_str)
    }))
    writer_test.write(example.SerializeToString())
'''
    

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

"\n\nfor index in index_list_for_test:\n    test_path = test_set[index]\n    \n    test_image = Image.open(test_path)\n    test_image = test_image.resize((img_height, img_width))\n    test_image = np.asarray(test_image)\n    test_image_str = test_image.tobytes()\n    \n    example = tf.train.Example(features=tf.train.Features(feature={\n        'test_image' : _bytes_feature(test_image_str)\n    }))\n    writer_test.write(example.SerializeToString())\n"

In [10]:
writer_train.close()
#writer_test.close()